#### IMPORTAMOS LIBRERIAS

In [154]:
from optparse import Option
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import random
from time import sleep
import pandas as pd
import numpy as np

#### FUNCION PARA OBTENER CREDENCIALES

In [155]:
def get_credentials():
    credentials = open('credentials.txt').readline().strip().split(',')
    user = credentials[0]
    password = credentials[1]
    
    return user, password

#### CREAMOS EL DRIVER

In [156]:

def get_driver():
    options = Options()
    #options.add_argument(
    #    "user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36"
    #)
    options.add_argument("--disable-notifications")
    options.add_argument("--headles")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-software-rasterizer")
    options.add_argument('--log-level=3')

    options.add_experimental_option('excludeSwitches', ['enable-logging'])


    driver = webdriver.Chrome('./chromedriver.exe', options = options)
    driver.maximize_window()
    driver.get('https://www.instagram.com/')
    
    
    sleep(random.uniform(3,4)) 

    return driver

#### FUNCIONES PRINCIPALES PARA SCRAPEAR

In [157]:
def cleaning_text(text):
    clean_text = text.replace('\n', ' ').replace('\r', ' ').replace('\t',' ').replace(',','').replace('~','').strip()
    return clean_text


def logging_instagram(driver):
    username, password = get_credentials()
    username_input = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@name="username"]'))
        )
    password_input =  WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@name="password"]'))
        )
    username_input.send_keys(username)
    password_input.send_keys(password)

    sleep(random.uniform(2,3)) 

    button = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//button[@type="submit"]'))
        )
    button.click()

    sleep(random.uniform(3,4))


def search_instagram(driver, HASHTAG):
    search_input =  WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@placeholder="Busca"]'))
        )
    search_input.send_keys(HASHTAG)

    sleep(random.uniform(3,4))

    search_link = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, f'//a[@href="/explore/tags/{HASHTAG.replace("#", "")}/"]'))
        )
    search_link.click()

    sleep(random.uniform(4,5))


def scraping_instagram(driver):

    data = []

    # Scrolling
    for i in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(random.uniform(4,5))
    
    posts = driver.find_elements(By.XPATH, '//article//a')
    for post in posts:
        try:
            post.click()      
            
            user = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//article[@role="presentation"]//h2//a'))).text
            description = cleaning_text(driver.find_element(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span').text)
            
            try:
                hashtags = driver.find_elements(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span/a[@class != "notranslate"]')
                hashtags = [cleaning_text(h.text) for h in hashtags]
            except:
                hashtags = []
            
            try:
                labels = driver.find_elements(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span/a[@class="notranslate"]')
                labels = [cleaning_text(l.text) for l in labels]
            except: 
                labels= []

            try:
                likes = driver.find_element(By.XPATH, '//article[@role="presentation"]//a[@class="zV_Nj"]/span').text
            except:
                likes = 0

            data.append({
                "user": user,
                "likes": likes,
                "labels":labels,
                "hashtag":hashtags,
                "description": description
            })

            # We use driver here, because post is limited to their scope 'a'
            close_button = WebDriverWait(driver, 10).until(   
                EC.presence_of_element_located((By.XPATH, '//div[@class="NOTWr"]/button'))
            )
            sleep(random.uniform(1,2))
            close_button.click()

        except Exception as e:
            continue
        
    df = pd.DataFrame(data)
    return df

def save_dataframe_to_csv(df):
    df.to_csv(r'data.csv', header=True, index=True, sep='~')

#### FUNCION TRIGGER QUE LANZA EL PROCESO

In [158]:
def run_scraping_process():
    HASHTAG = '#cuzco'
    driver = get_driver()
    logging_instagram(driver)
    search_instagram(driver, HASHTAG)
    df = scraping_instagram(driver)
    save_dataframe_to_csv(df)
    return df

In [159]:
data = run_scraping_process()
data

C:\Users\AlonsoPC\AppData\Local\Temp\ipykernel_2944\2026090804.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', options = options)


,user,likes,labels,hashtag,description
0,cusco.expedition,313,[@iamemanuelrocha],"[#cuscoexpedition, #cusco, #peru, #nomad, #and...",🇵🇪 Nuestras aventuras van más allá de simples ...
1,darwinrengifo,171,[],"[#2022, #happynewyear, #newyear, #añonuevo, #c...",1ER DÍA DEL AÑO 2022🗒✅ . Quedan 364 páginas má...
2,andres_cantos95,205,[],"[#cuzco, #peru, #sacsayhuaman, #travel]",Sacsayhuamán es la segunda edificación inca ma...
3,pahfm,111,[],"[#peru, #cuzco, #cuscodestinos, #cuscoentusojo...",Pensando que horas o mal de altitude vai passa...
4,cusco.expedition,312,[@ode_ruiz],"[#cuscoexpedition, #earthpix, #planetearth, #f...",🇵🇪 Hay tanto para ver y hacer en Cusco que es ...
5,luzdenazareth.sp,245,[],"[#cusco, #cuzco, #peru, #cuscoperu, #viaje, #m...",Siempre es grato volver a tan maravilloso luga...
6,turismo_machupicchu,482,[@luanaalully],"[#viajesamantra, #peru, #cusco, #lima, #travel...",Enjoy the real spirit of Machu Picchu. . . . :...
7,cusco.expedition,186,[@zn_gabyc],"[#cuscoexpedition, #cusco, #cuzco, #peru, #mac...",🇵🇪 Machu Picchu es un lugar increíble para con...
8,leila_barlow1,165,[],"[#peru, #lima, #travel, #machupicchu, #cuzco, ...",Best begging of the day with vanilla latte and...
9,superlopas_cusco,0,[],"[#cusco, #cuzco, #Delivery]",🛑🔥𝗤𝘂é 𝗱𝗲𝗹𝗶𝗰𝗶𝗮! 𝗣𝗶𝘇𝘇𝗮 𝗮 𝘁𝘂 𝗴𝘂𝘀𝘁𝗼 𝘆 𝗿𝗲𝗰𝗶é𝗻 𝗵𝗲𝗰𝗵𝗮...
